In [1]:
##import libraries
import pandas as pd
import numpy as np
import os
import glob
from functools import reduce
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import ast
import plotly.express as px

In [2]:
##path to our dataset
root_path = 'C:\\Users\\mulug\\OneDrive\\Documents\\Datasets\\'

In [9]:
def data_extraction(year, path, enc = 'utf-8'):
    
    ##list of target drug names
    target_drugs = ['xenical', 'alli', 'orlistat', 'liraglutide', 'saxenda']
    
    ##path to q1 ascii files
    q1_path = path + str(year)+"/faers_ascii_"+str(year)+"q1" + "/ASCII/"

    ##read demo data
    q1_DEMO = pd.read_csv(q1_path+f'DEMO{year%100}Q1.txt', delimiter = '$', dtype = str, encoding = enc)

    ##read drug data
    q1_DRUG = pd.read_csv(q1_path+f'DRUG{year%100}Q1.txt', delimiter = '$', dtype = str, encoding = enc)

    ##read indication data
    q1_INDI = pd.read_csv(q1_path+f'INDI{year%100}Q1.txt', delimiter = '$', dtype = str, encoding = enc)

    ##read outcome data
    q1_OUTC = pd.read_csv(q1_path+f'OUTC{year%100}Q1.txt', delimiter = '$', dtype = str, encoding = enc)

    ##read reaction data
    q1_REAC = pd.read_csv(q1_path+f'REAC{year%100}Q1.txt', delimiter = '$', dtype = str, encoding = enc)
    
    ##read therapy data
    q1_THER = pd.read_csv(q1_path+f'THER{year%100}Q1.txt', delimiter = '$', dtype = str, encoding = enc)
    
    #DEMO data copy
    merged_df1 = q1_DEMO.copy()
    
    ##aggregate multiple outcomes per report into a list
    outcome_agg = q1_OUTC.groupby("primaryid")["outc_cod"].apply(list).reset_index()
    
    ##merge the aggregated outcome data into merged_df1
    merged_df1 = merged_df1.merge(outcome_agg, on = ['primaryid'], how = 'left')
    
    ##aggregate multiple reactions per report into a list
    reac_agg = q1_REAC.groupby("primaryid")["pt"].apply(list).reset_index()
    
    ##merge the aggregated reaction data to merged_df1
    merged_df1 = merged_df1.merge(reac_agg, on = ['primaryid'], how = 'left')
    
    ##filter the target drugs
    q1_DRUG_filtered = q1_DRUG.loc[q1_DRUG['drugname'].str.lower().isin(target_drugs)]


    # Merge drug with indication using primaryid, caseid and drug_seq ↔ indi_drug_seq
    drug_indi = q1_DRUG_filtered.merge(q1_INDI, left_on=["primaryid", "caseid", "drug_seq"], right_on=["primaryid", "caseid", "indi_drug_seq"], how="left")
    
    # Merge drug_indi with q1_THER using primaryid, caseid and drug_seq ↔ dsg_drug_seq
    drug_indi_ther = drug_indi.merge(q1_THER, left_on=["primaryid", "caseid", "drug_seq"], right_on=["primaryid", "caseid", "dsg_drug_seq"], how="left")
    
    ##merge the drug_indi_ther to merged_df1
    merged_df1 = merged_df1.merge(drug_indi_ther, on = ['primaryid', 'caseid'], how = 'inner')
    
    #######################
    
    ##path to q2 ascii files
    q2_path = path + str(year)+"/faers_ascii_"+str(year)+"q2" + "/ASCII/"

    ##read demo data
    q2_DEMO = pd.read_csv(q2_path+f'DEMO{year%100}Q2.txt', delimiter = '$', dtype = str, encoding = enc)

    ##read drug data
    q2_DRUG = pd.read_csv(q2_path+f'DRUG{year%100}Q2.txt', delimiter = '$', dtype = str, encoding = enc)

    ##read indication data
    q2_INDI = pd.read_csv(q2_path+f'INDI{year%100}Q2.txt', delimiter = '$', dtype = str, encoding = enc)

    ##read outcome data
    q2_OUTC = pd.read_csv(q2_path+f'OUTC{year%100}Q2.txt', delimiter = '$', dtype = str, encoding = enc)

    ##read reaction data
    q2_REAC = pd.read_csv(q2_path+f'REAC{year%100}Q2.txt', delimiter = '$', dtype = str, encoding = enc)
    
    ##read therapy data
    q2_THER = pd.read_csv(q2_path+f'THER{year%100}Q2.txt', delimiter = '$', dtype = str, encoding = enc)
    

    #DEMO data copy
    merged_df2 = q2_DEMO.copy()
    
    ##aggregate multiple outcomes per report into a list
    outcome_agg = q2_OUTC.groupby("primaryid")["outc_cod"].apply(list).reset_index()
    
    ##merge the aggregated outcome data into merged_df2
    merged_df2 = merged_df2.merge(outcome_agg, on = ['primaryid'], how = 'left')
    
    ##aggregate multiple reactions per report into a list
    reac_agg = q2_REAC.groupby("primaryid")["pt"].apply(list).reset_index()
    
    ##merge the aggregated reaction data to merged_df2
    merged_df2 = merged_df2.merge(reac_agg, on = ['primaryid'], how = 'left')
    
    ##filter the target drugs
    q2_DRUG_filtered = q2_DRUG.loc[q2_DRUG['drugname'].str.lower().isin(target_drugs)]


    # Merge drug with indication using primaryid, caseid and drug_seq ↔ indi_drug_seq
    drug_indi = q2_DRUG_filtered.merge(q2_INDI, left_on=["primaryid", "caseid", "drug_seq"], right_on=["primaryid", "caseid", "indi_drug_seq"], how="left")
    
    # Merge drug_indi with q2_THER using primaryid, caseid and drug_seq ↔ dsg_drug_seq
    drug_indi_ther = drug_indi.merge(q2_THER, left_on=["primaryid", "caseid", "drug_seq"], right_on=["primaryid", "caseid", "dsg_drug_seq"], how="left")
    
    ##merge the drug_indi_ther to merged_df2
    merged_df2 = merged_df2.merge(drug_indi_ther, on = ['primaryid', 'caseid'], how = 'inner')

    
    ########################
    
    ##path to q3 ascii files
    q3_path = path + str(year)+"/faers_ascii_"+str(year)+"q3" + "/ASCII/"

    ##read demo data
    q3_DEMO = pd.read_csv(q3_path+f'DEMO{year%100}Q3.txt', delimiter = '$', dtype = str, encoding = enc)

    ##read drug data
    q3_DRUG = pd.read_csv(q3_path+f'DRUG{year%100}Q3.txt', delimiter = '$', dtype = str, encoding = enc)

    ##read indication data
    q3_INDI = pd.read_csv(q3_path+f'INDI{year%100}Q3.txt', delimiter = '$', dtype = str, encoding = enc)

    ##read outcome data
    q3_OUTC = pd.read_csv(q3_path+f'OUTC{year%100}Q3.txt', delimiter = '$', dtype = str, encoding = enc)

    ##read reaction data
    q3_REAC = pd.read_csv(q3_path+f'REAC{year%100}Q3.txt', delimiter = '$', dtype = str, encoding = enc)
    
    ##read therapy data
    q3_THER = pd.read_csv(q3_path+f'THER{year%100}Q3.txt', delimiter = '$', dtype = str, encoding = enc)
    

    #DEMO data copy
    merged_df3 = q3_DEMO.copy()
    
    ##aggregate multiple outcomes per report into a list
    outcome_agg = q3_OUTC.groupby("primaryid")["outc_cod"].apply(list).reset_index()
    
    ##merge the aggregated outcome data into merged_df3
    merged_df3 = merged_df3.merge(outcome_agg, on = ['primaryid'], how = 'left')
    
    ##aggregate multiple reactions per report into a list
    reac_agg = q3_REAC.groupby("primaryid")["pt"].apply(list).reset_index()
    
    ##merge the aggregated reaction data to merged_df3
    merged_df3 = merged_df3.merge(reac_agg, on = ['primaryid'], how = 'left')
    
    ##filter the target drugs
    q3_DRUG_filtered = q3_DRUG.loc[q3_DRUG['drugname'].str.lower().isin(target_drugs)]


    # Merge drug with indication using primaryid, caseid and drug_seq ↔ indi_drug_seq
    drug_indi = q3_DRUG_filtered.merge(q3_INDI, left_on=["primaryid", "caseid", "drug_seq"], right_on=["primaryid", "caseid", "indi_drug_seq"], how="left")
    
    # Merge drug_indi with q3_THER using primaryid, caseid and drug_seq ↔ dsg_drug_seq
    drug_indi_ther = drug_indi.merge(q3_THER, left_on=["primaryid", "caseid", "drug_seq"], right_on=["primaryid", "caseid", "dsg_drug_seq"], how="left")
    
    ##merge the drug_indi_ther to merged_df3
    merged_df3 = merged_df3.merge(drug_indi_ther, on = ['primaryid', 'caseid'], how = 'inner')


    #######################
    
    ##path to q4 ascii files
    q4_path = path + str(year)+"/faers_ascii_"+str(year)+"Q4" + "/ASCII/"

    ##read demo data
    q4_DEMO = pd.read_csv(q4_path+f'DEMO{year%100}Q4.txt', delimiter = '$', dtype = str, encoding = enc)

    ##read drug data
    q4_DRUG = pd.read_csv(q4_path+f'DRUG{year%100}Q4.txt', delimiter = '$', dtype = str, encoding = enc)

    ##read indication data
    q4_INDI = pd.read_csv(q4_path+f'INDI{year%100}Q4.txt', delimiter = '$', dtype = str, encoding = enc)

    ##read outcome data
    q4_OUTC = pd.read_csv(q4_path+f'OUTC{year%100}Q4.txt', delimiter = '$', dtype = str, encoding = enc)

    ##read reaction data
    q4_REAC = pd.read_csv(q4_path+f'REAC{year%100}Q4.txt', delimiter = '$', dtype = str, encoding = enc)
    
    ##read therapy data
    q4_THER = pd.read_csv(q4_path+f'THER{year%100}Q4.txt', delimiter = '$', dtype = str, encoding = enc)
    

    #DEMO data copy
    merged_df4 = q4_DEMO.copy()
    
    ##aggregate multiple outcomes per report into a list
    outcome_agg = q4_OUTC.groupby("primaryid")["outc_cod"].apply(list).reset_index()
    
    ##merge the aggregated outcome data into merged_df4
    merged_df4 = merged_df4.merge(outcome_agg, on = ['primaryid'], how = 'left')
    
    ##aggregate multiple reactions per report into a list
    reac_agg = q4_REAC.groupby("primaryid")["pt"].apply(list).reset_index()
    
    ##merge the aggregated reaction data to merged_df4
    merged_df4 = merged_df4.merge(reac_agg, on = ['primaryid'], how = 'left')
    
    ##filter the target drugs
    q4_DRUG_filtered = q4_DRUG.loc[q4_DRUG['drugname'].str.lower().isin(target_drugs)]


    # Merge drug with indication using primaryid, caseid and drug_seq ↔ indi_drug_seq
    drug_indi = q4_DRUG_filtered.merge(q4_INDI, left_on=["primaryid", "caseid", "drug_seq"], right_on=["primaryid", "caseid", "indi_drug_seq"], how="left")
    
    # Merge drug_indi with q4_THER using primaryid, caseid and drug_seq ↔ dsg_drug_seq
    drug_indi_ther = drug_indi.merge(q4_THER, left_on=["primaryid", "caseid", "drug_seq"], right_on=["primaryid", "caseid", "dsg_drug_seq"], how="left")
    
    ##merge the drug_indi_ther to merged_df4
    merged_df4 = merged_df4.merge(drug_indi_ther, on = ['primaryid', 'caseid'], how = 'inner')

    ##return the full data for each year as list of 
    ##quarterly dfs
    return [merged_df1, merged_df2, merged_df3, merged_df4]

In [10]:
##yearly extracted data for 2024
dfs_2024 = data_extraction(2024, root_path)

##concatenate dfs
data_2024 = pd.concat(dfs_2024, ignore_index = True)

##display data
data_2024

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,...,dose_unit,dose_form,dose_freq,indi_drug_seq,indi_pt,dsg_drug_seq,start_dt,end_dt,dur,dur_cod
0,1131845744,11318457,44,F,20150101,20240125,20150729,20240206,EXP,NaN,...,NaN,NaN,NaN,50,Product used for unknown indication,50,20231201,NaN,NaN,NaN
1,1564086814,15640868,14,F,NaN,20240111,20181120,20240118,EXP,NaN,...,NaN,Capsule,NaN,119,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
2,1564086814,15640868,14,F,NaN,20240111,20181120,20240118,EXP,NaN,...,NaN,Capsule,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,157756714,15775671,4,F,20190101,20240205,20181231,20240213,EXP,NaN,...,MG,NaN,QD,20,Weight decreased,NaN,NaN,NaN,NaN,NaN
4,168364652,16836465,2,F,NaN,20240305,20190921,20240318,EXP,NaN,...,NaN,NaN,NaN,16,Diabetes mellitus,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790,247864171,24786417,1,I,20240801,20241218,20241230,20241230,EXP,NaN,...,NaN,Solution for injection,NaN,2,Weight control,2,202408,NaN,NaN,NaN
1791,247901862,24790186,2,F,NaN,20241226,20241230,20241231,PER,NaN,...,NaN,Capsule,NaN,1,Weight decreased,1,20241224,NaN,NaN,NaN
1792,247912301,24791230,1,I,20241214,20241219,20241230,20241230,EXP,NaN,...,MG,Solution for injection,QD,1,Obesity,NaN,NaN,NaN,NaN,NaN
1793,247913931,24791393,1,I,20220201,20241220,20241231,20241231,EXP,NaN,...,MG,Solution for injection,QD,1,Obesity,1,202202,202203,NaN,NaN


In [11]:
##yearly data for 2023   
dfs_2023 = data_extraction(2023, root_path)

##concatenate the returned dfs
data_2023 = pd.concat(dfs_2023, ignore_index = True)

##display data
data_2023

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,...,dose_unit,dose_form,dose_freq,indi_drug_seq,indi_pt,dsg_drug_seq,start_dt,end_dt,dur,dur_cod
0,111834804,11183480,4,F,20090401,20230126,20150612,20230207,EXP,NaN,...,NaN,NaN,NaN,2,Obesity,2,200903,200906,3,MON
1,1327603118,13276031,18,F,NaN,20230110,20170228,20230113,EXP,NaN,...,NaN,Capsule,NaN,72,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
2,148792524,14879252,4,F,NaN,20230104,20180511,20230106,EXP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1522229623,15222296,23,F,20180101,20221230,20180731,20230106,EXP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,162103223,16210322,3,F,NaN,20221221,20190418,20230104,EXP,NaN,...,NaN,NaN,NaN,21,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1905,233450431,23345043,1,I,20231210,20231219,20231228,20231228,EXP,NaN,...,NaN,Capsule,NaN,1,Product used for unknown indication,1,20231210,20231214,4,DAY
1906,233455541,23345554,1,I,NaN,20231204,20231228,20231228,EXP,NaN,...,MG,NaN,QD,2,Type 2 diabetes mellitus,NaN,NaN,NaN,NaN,NaN
1907,233511921,23351192,1,I,20230731,20231215,20231229,20231229,EXP,NaN,...,NaN,Solution for injection,NaN,1,Product used for unknown indication,1,20230619,20230801,43,DAY
1908,233517301,23351730,1,I,20210101,20231218,20231229,20231229,EXP,NaN,...,NaN,Solution for injection,NaN,1,Weight control,1,2020,2021,NaN,NaN


In [12]:
##yearly data for 2022
dfs_2022 = data_extraction(2022, root_path)

##concatenate the returned dfs
data_2022 = pd.concat(dfs_2022, ignore_index = True)

##display data
data_2022

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,...,dose_unit,dose_form,dose_freq,indi_drug_seq,indi_pt,dsg_drug_seq,start_dt,end_dt,dur,dur_cod
0,1277830944,12778309,44,F,NaN,20220119,20160926,20220126,EXP,NaN,...,NaN,Capsule,NaN,122,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
1,128620824,12862082,4,F,20150515,20220203,20161019,20220210,EXP,NaN,...,MG,NaN,QD,10,Diabetes mellitus,NaN,NaN,NaN,NaN,NaN
2,1352573514,13525735,14,F,20170417,20220103,20170509,20220118,EXP,NaN,...,NaN,NaN,NaN,20,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
3,1494356917,14943569,17,F,20160501,20220203,20180528,20220209,EXP,NaN,...,NaN,NaN,NaN,49,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
4,1522229618,15222296,18,F,20180101,20220310,20180731,20220324,EXP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,217920111,21792011,1,I,20221025,20221219,20221229,20221229,EXP,SE-MPA-2022-022502,...,MG,Solution for injection,QD,NaN,NaN,1,NaN,20221128,NaN,NaN
1461,217920111,21792011,1,I,20221025,20221219,20221229,20221229,EXP,SE-MPA-2022-022502,...,MG,Solution for injection,QD,2,Obesity,2,20221014,NaN,NaN,NaN
1462,217982491,21798249,1,I,NaN,20221219,20221230,20221230,EXP,NaN,...,MG,Solution for injection,QD,3,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
1463,217982491,21798249,1,I,NaN,20221219,20221230,20221230,EXP,NaN,...,NaN,Solution for injection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
##yearly data for 2021
dfs_2021 = data_extraction(2021, root_path)

##concatenate the returned dfs 
data_2021 = pd.concat(dfs_2021, ignore_index = True)

##display data
data_2021

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,...,dose_unit,dose_form,dose_freq,indi_drug_seq,indi_pt,dsg_drug_seq,start_dt,end_dt,dur,dur_cod
0,136618683,13661868,3,F,NaN,20210315,20170616,20210326,EXP,NaN,...,MG,CAPSULE,NaN,2,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
1,137730739,13773073,9,F,2009,20210205,20170720,20210216,EXP,NaN,...,NaN,NaN,NaN,NaN,NaN,18,2020,NaN,NaN,NaN
2,148792523,14879252,3,F,NaN,20210108,20180511,20210115,EXP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1522229610,15222296,10,F,2018,20201224,20180731,20210102,EXP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,152836843,15283684,3,F,2018,20210317,20180816,20210324,PER,NaN,...,MG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1585,202636771,20263677,1,I,NaN,20211221,20211231,20211231,EXP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1586,202640121,20264012,1,I,NaN,20211219,20211231,20211231,EXP,GB-MHRA-ADR 26338435,...,NaN,NaN,NaN,1,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
1587,202647141,20264714,1,I,20211209,20211224,20211231,20211231,EXP,NaN,...,MG,Injection,QD,2,Diabetes mellitus,2,20211130,20211202,3,DAY
1588,74735216,7473521,6,F,20090401,20211023,20100714,20211029,EXP,GB-MHRA-ADR 20650063,...,NaN,NaN,TID,3,Obesity,3,200903,200906,NaN,NaN


In [14]:
##yearly data for 2020
dfs_2020 = data_extraction(2020, root_path)

##concatenate the returned dfs
data_2020 = pd.concat(dfs_2020, ignore_index = True)

##display data
data_2020

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,...,dose_unit,dose_form,dose_freq,indi_drug_seq,indi_pt,dsg_drug_seq,start_dt,end_dt,dur,dur_cod
0,120963692,12096369,2,F,2007,20200326,20160220,20200331,EXP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1314962521,13149625,21,F,2017,20200218,20170125,20200302,EXP,NaN,...,NaN,NaN,NaN,NaN,NaN,17,2006,NaN,NaN,NaN
2,1352573513,13525735,13,F,20170417,20191223,20170509,20200106,EXP,NaN,...,NaN,NaN,NaN,21,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
3,147306292,14730629,2,F,20180226,20200227,20180406,20200309,EXP,GB-MHRA-ADR 24296496,...,NaN,SOLUTION FOR INJECTION,NaN,1,Diabetes mellitus,1,NaN,20180226,NaN,NaN
4,148324085,14832408,5,F,20080514,20200114,20180501,20200122,EXP,NaN,...,NaN,NaN,NaN,41,Weight decreased,41,2007,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1696,186787311,18678731,1,I,20201211,20201221,20201229,20201229,EXP,NaN,...,NaN,SOLUTION FOR INJECTION,NaN,1,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
1697,186830011,18683001,1,I,NaN,20201220,20201230,20201230,EXP,NaN,...,NaN,CAPSULE,NaN,1,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
1698,186830111,18683011,1,I,NaN,20201221,20201230,20201230,EXP,NaN,...,NaN,CAPSULE,NaN,1,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
1699,186896241,18689624,1,I,NaN,20201224,20201231,20201231,EXP,NaN,...,NaN,SOLUTION FOR INJECTION,NaN,1,Product used for unknown indication,1,201906,NaN,NaN,NaN


In [15]:
##five years data (2020-2024)
dfs_1 = [data_2020, data_2021, data_2022, data_2023, data_2024]

##merge the five years data into one df
five_yrs_data = pd.concat(dfs_1, ignore_index = True)

##display data
five_yrs_data

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,...,dose_unit,dose_form,dose_freq,indi_drug_seq,indi_pt,dsg_drug_seq,start_dt,end_dt,dur,dur_cod
0,120963692,12096369,2,F,2007,20200326,20160220,20200331,EXP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1314962521,13149625,21,F,2017,20200218,20170125,20200302,EXP,NaN,...,NaN,NaN,NaN,NaN,NaN,17,2006,NaN,NaN,NaN
2,1352573513,13525735,13,F,20170417,20191223,20170509,20200106,EXP,NaN,...,NaN,NaN,NaN,21,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
3,147306292,14730629,2,F,20180226,20200227,20180406,20200309,EXP,GB-MHRA-ADR 24296496,...,NaN,SOLUTION FOR INJECTION,NaN,1,Diabetes mellitus,1,NaN,20180226,NaN,NaN
4,148324085,14832408,5,F,20080514,20200114,20180501,20200122,EXP,NaN,...,NaN,NaN,NaN,41,Weight decreased,41,2007,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8456,247864171,24786417,1,I,20240801,20241218,20241230,20241230,EXP,NaN,...,NaN,Solution for injection,NaN,2,Weight control,2,202408,NaN,NaN,NaN
8457,247901862,24790186,2,F,NaN,20241226,20241230,20241231,PER,NaN,...,NaN,Capsule,NaN,1,Weight decreased,1,20241224,NaN,NaN,NaN
8458,247912301,24791230,1,I,20241214,20241219,20241230,20241230,EXP,NaN,...,MG,Solution for injection,QD,1,Obesity,NaN,NaN,NaN,NaN,NaN
8459,247913931,24791393,1,I,20220201,20241220,20241231,20241231,EXP,NaN,...,MG,Solution for injection,QD,1,Obesity,1,202202,202203,NaN,NaN


In [16]:
##yearly data for 2019
dfs_2019 = data_extraction(2019, root_path, 'latin-1')

##concatenate the returned dfs
data_2019 = pd.concat(dfs_2019, ignore_index = True)

##display data
data_2019

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,...,dose_unit,dose_form,dose_freq,indi_drug_seq,indi_pt,dsg_drug_seq,start_dt,end_dt,dur,dur_cod
0,105110067,10511006,7,F,NaN,20190111,20141010,20190122,PER,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,121928105,12192810,5,F,20160212,20190107,20160318,20190116,EXP,NaN,...,NaN,NaN,NaN,8,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
2,127370444,12737044,4,F,20160902,20190213,20160913,20190218,EXP,NaN,...,NaN,NaN,NaN,NaN,NaN,11,NaN,20160901,NaN,NaN
3,1302531011,13025310,11,F,201602,20190312,20161214,20190322,EXP,NaN,...,NaN,NaN,NaN,4,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
4,131601416,13160141,6,F,20120220,20190304,20170127,20190307,EXP,NaN,...,NaN,NaN,QD,71,Diabetes mellitus,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877,172138441,17213844,1,I,NaN,20191216,20191230,20191230,EXP,NaN,...,MG,INJECTION,QD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1878,172138441,17213844,1,I,NaN,20191216,20191230,20191230,EXP,NaN,...,MG,INJECTION,QD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1879,172138441,17213844,1,I,NaN,20191216,20191230,20191230,EXP,NaN,...,NaN,INJECTION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1880,172138441,17213844,1,I,NaN,20191216,20191230,20191230,EXP,NaN,...,MG,INJECTION,QD,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
##yearly data for 2018
dfs_2018 = data_extraction(2018, root_path)

##concatenate the returned dfs
data_2018 = pd.concat(dfs_2018, ignore_index = True)

##display data
data_2018

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,...,dose_unit,dose_form,dose_freq,indi_drug_seq,indi_pt,dsg_drug_seq,start_dt,end_dt,dur,dur_cod
0,121883812,12188381,2,F,20150408,20180108,20160317,20180108,EXP,NaN,...,MG,NaN,NaN,20,Type 2 diabetes mellitus,20,20130724,20131107,NaN,NaN
1,121895522,12189552,2,F,NaN,20180306,20160318,20180314,EXP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1258207112,12582071,12,F,20160629,20180314,20160722,20180321,EXP,NaN,...,NaN,NaN,NaN,5,Weight control,5,2013,NaN,NaN,NaN
3,130253109,13025310,9,F,201602,20180126,20161214,20180208,EXP,NaN,...,NaN,NaN,NaN,14,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
4,130594343,13059434,3,F,NaN,20180207,20161223,20180215,PER,NaN,...,NaN,NaN,NaN,1,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1724,157858001,15785800,1,I,20181101,NaN,20181212,20181212,DIR,NaN,...,NaN,NaN,NaN,1,Diabetes mellitus,1,20180416,20181101,NaN,NaN
1725,158634781,15863478,1,I,20180909,NaN,20181228,20181228,DIR,NaN,...,MG,NaN,QD,1,Diabetes mellitus,1,20180809,20181106,NaN,NaN
1726,826565868,8265658,68,F,201105,20181028,20111128,20181108,EXP,NaN,...,MG,NaN,NaN,54,Weight decreased,54,201708,NaN,NaN,NaN
1727,846873712,8468737,12,F,2003,20181127,20120320,20181205,EXP,NaN,...,MG,CAPSULE,QD,NaN,NaN,59,20030211,NaN,NaN,NaN


In [18]:
##yearly data for 2017
dfs_2017 = data_extraction(2017, root_path)

##concatenate the returned dfs
data_2017 = pd.concat(dfs_2017, ignore_index = True)

##display data
data_2017

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,...,dose_unit,dose_form,dose_freq,indi_drug_seq,indi_pt,dsg_drug_seq,start_dt,end_dt,dur,dur_cod
0,1015990810,10159908,10,F,201312,20170203,20140508,20170213,EXP,NaN,...,NaN,NaN,NaN,NaN,NaN,2,201402,NaN,NaN,NaN
1,104070864,10407086,4,F,2014,20170119,20140825,20170123,PER,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,117918928,11791892,8,F,NaN,20170118,20151201,20170119,PER,NaN,...,NaN,CAPSULE,NaN,10,Product used for unknown indication,10,2006,NaN,NaN,NaN
3,120839037,12083903,7,F,201602,20170221,20160217,20170307,EXP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,122400272,12240027,2,F,NaN,20160504,20160405,20170316,PER,NaN,...,NaN,NaN,NaN,1,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2365,143321261,14332126,1,I,NaN,20171218,20171228,20171228,EXP,NaN,...,NaN,SOLUTION FOR INJECTION,NaN,1,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
2366,143323591,14332359,1,I,NaN,20171223,20171228,20171228,EXP,NaN,...,NaN,NaN,NaN,1,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
2367,143375491,14337549,1,I,NaN,20171031,20171229,20171229,EXP,NaN,...,MG,SOLUTION FOR INJECTION,NaN,1,Weight decreased,NaN,NaN,NaN,NaN,NaN
2368,143375491,14337549,1,I,NaN,20171031,20171229,20171229,EXP,NaN,...,MG,SOLUTION FOR INJECTION,QD,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
##yearly data for 2016
dfs_2016 = data_extraction(2016, root_path)

##concatenate the returned dfs
data_2016 = pd.concat(dfs_2016, ignore_index = True)

##display data
data_2016

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,...,dose_unit,dose_form,dose_freq,indi_drug_seq,indi_pt,dsg_drug_seq,start_dt,end_dt,dur,dur_cod
0,100189298,10018929,8,F,20121203,20160302,20140318,20160307,EXP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,105123726,10512372,6,F,20140913,20160128,20141010,20160209,EXP,NaN,...,NaN,NaN,NaN,13,Product used for unknown indication,13,2012,NaN,NaN,NaN
2,111576402,11157640,2,F,NaN,20160225,20150603,20160309,EXP,NaN,...,DF,NaN,TID,2,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
3,111697402,11169740,2,F,NaN,20160225,20150606,20160309,EXP,NaN,...,NaN,NaN,NaN,2,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
4,111736652,11173665,2,F,NaN,20160225,20150609,20160309,EXP,US-FDA-7468998,...,DF,NaN,TID,2,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2483,130756601,13075660,1,I,20161201,20161202,20161230,20161230,PER,NaN,...,NaN,NaN,NaN,3,Weight decreased,NaN,NaN,NaN,NaN,NaN
2484,81695184,8169518,4,F,NaN,20160622,20111005,20161104,PER,NaN,...,NaN,CAPSULE,NaN,1,Weight loss diet,1,NaN,20110921,NaN,NaN
2485,81695184,8169518,4,F,NaN,20160622,20111005,20161104,PER,NaN,...,NaN,CAPSULE,NaN,NaN,NaN,2,2015,2015,NaN,NaN
2486,81742183,8174218,3,F,2011,20111108,20111010,20161026,EXP,NaN,...,MG,INJECTION,QD,8,Diabetes mellitus,NaN,NaN,NaN,NaN,NaN


In [20]:
##yearly data for 2015
dfs_2015 = data_extraction(2015, root_path)

##concatenate the returned dfs
data_2015 = pd.concat(dfs_2015, ignore_index = True)

##display data
data_2015

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,...,dose_unit,dose_form,dose_freq,indi_drug_seq,indi_pt,dsg_drug_seq,start_dt,end_dt,dur,dur_cod
0,100593752,10059375,2,F,20130509,20140407,20140404,20150210,EXP,NaN,...,NaN,CAPSULE,BID,1,Product used for unknown indication,1,20130510,20130530,NaN,NaN
1,100600032,10060003,2,F,NaN,20131121,20140404,20150210,EXP,NaN,...,NaN,CAPSULE,NaN,1,Weight decreased,NaN,NaN,NaN,NaN,NaN
2,100601192,10060119,2,F,NaN,20141218,20140404,20150107,EXP,NaN,...,NaN,CAPSULE,NaN,1,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
3,102706412,10270641,2,F,20140414,20140617,20140701,20140701,EXP,GB-MHRA-ADR 22571245,...,MG,NaN,TID,5,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
4,103611546,10361154,6,F,20140724,20150304,20140804,20150312,EXP,NaN,...,NaN,NaN,NaN,8,Type 2 diabetes mellitus,8,20140109,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2389,118830901,11883090,1,I,NaN,20151229,20151231,20151231,EXP,NaN,...,NaN,NaN,NaN,1,Weight loss diet,NaN,NaN,NaN,NaN,NaN
2390,118849651,11884965,1,I,20150922,NaN,20151231,20151231,DIR,NaN,...,MG,NaN,QD,NaN,NaN,1,20150901,20150925,NaN,NaN
2391,91003424,9100342,4,F,20130125,20151204,20130215,20151211,EXP,NaN,...,NaN,NaN,NaN,2,Diabetes mellitus,2,201012,NaN,NaN,NaN
2392,99153282,9915328,2,F,20110617,20110607,20140221,20151118,EXP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
##five years data (2015-2019)
dfs_2 = [data_2015, data_2016, data_2017, data_2018, data_2019]

##merge the dfs
five_yrs_data_2 = pd.concat(dfs_2, ignore_index = True)

##dispaly data
five_yrs_data_2

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,...,dose_unit,dose_form,dose_freq,indi_drug_seq,indi_pt,dsg_drug_seq,start_dt,end_dt,dur,dur_cod
0,100593752,10059375,2,F,20130509,20140407,20140404,20150210,EXP,NaN,...,NaN,CAPSULE,BID,1,Product used for unknown indication,1,20130510,20130530,NaN,NaN
1,100600032,10060003,2,F,NaN,20131121,20140404,20150210,EXP,NaN,...,NaN,CAPSULE,NaN,1,Weight decreased,NaN,NaN,NaN,NaN,NaN
2,100601192,10060119,2,F,NaN,20141218,20140404,20150107,EXP,NaN,...,NaN,CAPSULE,NaN,1,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
3,102706412,10270641,2,F,20140414,20140617,20140701,20140701,EXP,GB-MHRA-ADR 22571245,...,MG,NaN,TID,5,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
4,103611546,10361154,6,F,20140724,20150304,20140804,20150312,EXP,NaN,...,NaN,NaN,NaN,8,Type 2 diabetes mellitus,8,20140109,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10858,172138441,17213844,1,I,NaN,20191216,20191230,20191230,EXP,NaN,...,MG,INJECTION,QD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10859,172138441,17213844,1,I,NaN,20191216,20191230,20191230,EXP,NaN,...,MG,INJECTION,QD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10860,172138441,17213844,1,I,NaN,20191216,20191230,20191230,EXP,NaN,...,NaN,INJECTION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10861,172138441,17213844,1,I,NaN,20191216,20191230,20191230,EXP,NaN,...,MG,INJECTION,QD,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
##merge the ten years data
ten_yrs_data = pd.concat([five_yrs_data, five_yrs_data_2], ignore_index = True)

##display data
ten_yrs_data

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,...,dose_unit,dose_form,dose_freq,indi_drug_seq,indi_pt,dsg_drug_seq,start_dt,end_dt,dur,dur_cod
0,120963692,12096369,2,F,2007,20200326,20160220,20200331,EXP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1314962521,13149625,21,F,2017,20200218,20170125,20200302,EXP,NaN,...,NaN,NaN,NaN,NaN,NaN,17,2006,NaN,NaN,NaN
2,1352573513,13525735,13,F,20170417,20191223,20170509,20200106,EXP,NaN,...,NaN,NaN,NaN,21,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN
3,147306292,14730629,2,F,20180226,20200227,20180406,20200309,EXP,GB-MHRA-ADR 24296496,...,NaN,SOLUTION FOR INJECTION,NaN,1,Diabetes mellitus,1,NaN,20180226,NaN,NaN
4,148324085,14832408,5,F,20080514,20200114,20180501,20200122,EXP,NaN,...,NaN,NaN,NaN,41,Weight decreased,41,2007,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19319,172138441,17213844,1,I,NaN,20191216,20191230,20191230,EXP,NaN,...,MG,INJECTION,QD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19320,172138441,17213844,1,I,NaN,20191216,20191230,20191230,EXP,NaN,...,MG,INJECTION,QD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19321,172138441,17213844,1,I,NaN,20191216,20191230,20191230,EXP,NaN,...,NaN,INJECTION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19322,172138441,17213844,1,I,NaN,20191216,20191230,20191230,EXP,NaN,...,MG,INJECTION,QD,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
##save to csv file

##file path
csv_file = root_path+'Ten_years_data.csv'

##save the file
ten_yrs_data.to_csv(csv_file, index = False)